In [1]:
%matplotlib widget
import numpy as np
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random

import matplotlib.pyplot as plt

import re

from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

from joblib import Parallel, delayed

import multiprocessing
from datetime import datetime

from torch.utils.data import DataLoader, Dataset
from IPython.display import clear_output

In [2]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:1")
device

device(type='cuda', index=1)

In [3]:
torch.manual_seed(1010101011)
random.seed(1010101011)

In [4]:
ftrainset = open("../dataset/train.txt", encoding='utf8')
trainset = [slowo.replace("\n", "") for slowo in ftrainset.readlines() if len(slowo) < 22]
ftrainset.close()

ftestset = open("../dataset/test.txt", encoding='utf8')
testset = [slowo.replace("\n", "") for slowo in ftestset.readlines()]
ftestset.close()

In [5]:
# trainset = ["abc", "abcd", "abcccc", "bcde"]
# testset = []

In [6]:
print("Train: ", len(trainset))
print("Test: ", len(testset))

Train:  169668
Test:  8931


In [7]:
chartoidx = {}

cnt = 0

longestword = 0

chartoidx["<EMPTY>"] = 0

for slowo in tqdm_notebook(trainset+testset):  
    for litera in slowo:
        if litera not in list(chartoidx.keys()):
            chartoidx[litera] = cnt+1
            cnt = cnt + 1
            
    if len(slowo) > longestword:
        longestword = len(slowo)
            
vocabsize = len(list(chartoidx.keys()))

In [8]:
longestword

19

In [9]:
chartoidx

{'<EMPTY>': 0,
 'p': 1,
 'i': 2,
 'k': 3,
 'u': 4,
 's': 5,
 '8': 6,
 '0': 7,
 'a': 8,
 'n': 9,
 'e': 10,
 'c': 11,
 'A': 12,
 '1': 13,
 '2': 14,
 '3': 15,
 '4': 16,
 '5': 17,
 '6': 18,
 '7': 19,
 '9': 20,
 'l': 21,
 't': 22,
 'g': 23,
 'h': 24,
 'd': 25,
 'm': 26,
 'r': 27,
 'M': 28,
 'I': 29,
 'S': 30,
 'K': 31,
 'J': 32,
 'o': 33,
 'P': 34,
 'j': 35,
 'z': 36,
 'B': 37,
 'U': 38,
 'b': 39,
 'f': 40,
 'w': 41,
 'v': 42,
 'y': 43,
 'H': 44,
 'L': 45,
 'O': 46,
 'T': 47,
 'E': 48,
 'R': 49,
 'N': 50,
 'x': 51,
 'G': 52,
 'q': 53,
 'D': 54,
 'Q': 55,
 'Z': 56,
 'V': 57,
 'C': 58,
 'X': 59,
 'W': 60,
 'Y': 61,
 'F': 62,
 '!': 63,
 '-': 64,
 ' ': 65,
 '@': 66,
 '.': 67,
 '_': 68,
 '/': 69,
 '[': 70,
 ']': 71,
 '~': 72,
 '#': 73,
 '*': 74,
 '(': 75,
 ')': 76,
 '+': 77,
 '`': 78,
 '$': 79,
 ',': 80,
 '%': 81,
 '^': 82,
 '=': 83,
 ';': 84,
 '&': 85,
 '?': 86,
 'ł': 87}

In [10]:
slowo = trainset[0]; slowo

'pikus80'

In [11]:
window = 5

slowka = ["abc", "defg"]
slowkaidx = [0, 0, 0, 1, 1, 1, 1]

def getitem(slowa, index, window, slowkaidx):
    slowo = slowa[slowkaidx[index]]
    
    idx = slowkaidx[index]
    currindex = 0
    
    while idx == slowkaidx[index]:
        currindex += 1
        idx = slowkaidx[index-currindex]
    currindex -= 1
    
    ins = []
    out = []
    
    ins = [litera for litera in slowo[:currindex+1]]
    for __ in range(window-len(ins)):
        ins.append("<")
    
    if currindex<len(slowo)-1:
        out = [slowo[currindex+1]]
    else:
        out = ["<"]
    
    print(ins, out)

for i in range(len(slowkaidx)):
    getitem(slowka, i, window, slowkaidx)   

['a', '<', '<', '<', '<'] ['b']
['a', 'b', '<', '<', '<'] ['c']
['a', 'b', 'c', '<', '<'] ['<']
['d', '<', '<', '<', '<'] ['e']
['d', 'e', '<', '<', '<'] ['f']
['d', 'e', 'f', '<', '<'] ['g']
['d', 'e', 'f', 'g', '<'] ['<']


In [12]:
class MyDataset(Dataset):
    def __init__(self, slowa, chartoidx, longestword, padding):
        self.slowa = slowa
        self.chartoidx = chartoidx
        self.longestword = longestword
        self.padding = padding
        self.indexpos = []
        
        for idd, slowo in enumerate(slowa):
            for letter in slowo:
                self.indexpos.append(idd)
        
    def __len__(self):
        return len(self.indexpos)
        
    def __getitem__(self, index):
        literyx = []
        literyy = []
        
        slowo = self.slowa[self.indexpos[index]]
        
        
        idx = self.indexpos[index]
        currindex = 0
    
        while idx == self.indexpos[index]:
            currindex += 1
            idx = self.indexpos[index-currindex]
        currindex -= 1
        
        
        literyx = [self.chartoidx[litera] for litera in slowo[:currindex+1]]
        for __ in range(self.longestword-len(literyx)):
            literyx.append(self.chartoidx["<EMPTY>"])
    
        if currindex<len(slowo)-1:
            literyy = [self.chartoidx[slowo[currindex+1]]]
        else:
            literyy = [self.chartoidx["<EMPTY>"]]
                
        return np.array(literyx, dtype="float32"), np.array(literyy, dtype="long")

In [13]:
DS_train = MyDataset(trainset, chartoidx, longestword, 0)
DS_test = MyDataset(testset, chartoidx, longestword, 0)

In [14]:
BS = 40000

In [15]:
DS_train.__getitem__(13)

(array([12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.], dtype=float32), array([12]))

In [16]:
DL_train = DataLoader(dataset=DS_train, batch_size=BS, num_workers=5)
DL_test = DataLoader(dataset=DS_test, batch_size=BS, num_workers=5)

In [17]:
class CharacterLSTM(nn.Module):
    def __init__(self, vocabsize, lstmlayers, hiddensize):
        super(CharacterLSTM, self).__init__()
        
        
        ## WARSTWY
        self.embd = nn.Embedding(vocabsize, vocabsize)
        self.LSTM1 = nn.GRU(vocabsize, hiddensize, lstmlayers, batch_first=True, bidirectional=True)
        self.linear_ins = nn.Linear(2*hiddensize, vocabsize)
#         self.linear_outs = nn.Linear(2*vocabsize, vocabsize)
        
        ## TRICKY LAYERS
#         self.norm = nn.BatchNorm1d(windowsize)
#         self.norm = nn.LayerNorm(2*vocabsize)
#         self.norm2 = nn.LayerNorm(vocabsize)
        self.drop = nn.Dropout(p=0.1)
        
        ## OUTS
        self.softmax = nn.LogSoftmax(dim=1)
        
        
#         self.init_weights()
        
    def init_weights(self):
        initrange = 0.1
        self.embd.weight.data.uniform_(-initrange, initrange)
        self.linear_ins.bias.data.zero_()
        self.linear_ins.weight.data.uniform_(-initrange, initrange)
#         self.linear_outs.bias.data.zero_()
#         self.linear_outs.weight.data.uniform_(-initrange, initrange)
        
    def forward(self, x, hidden, NLL=True):
#         print(x.shape)
        
        # WEJSCIE
        y0 = self.embd(x)
#         y0  = self.drop(y0)
#         y0 = self.EmbeddingDrop(y0, p=0.2)

        
        # DROP
#         if drop>0:
#             y0 = self.EmbeddingDrop(y0, p=drop)
        
        # LSTM
        y, h1_ = self.LSTM1(y0, hidden)
        
        # TRICKY
#         y = self.norm(y)
        
        y = self.drop(y)
        
        # LINEAR OUT 1
        y = self.linear_ins(y)
#         y = self.norm(y)
#         y = self.linear_ins(y)
        
#         # TRICKY
#         y = self.norm2(y)
    
#         # TRICKY 2
#         y = self.drop(y)
        
        
#         # LINEAR OUT 2
#         y = F.relu(self.linear_outs(y))
      
#         y = self.norm2(y)

#         print(y, y.shape)
        
        if NLL:
            y = self.softmax(y[:,-1])
        
        # zwrot
        return y, h1_
    
#     def EmbeddingDrop(self, dataset, p=0.1):
#         w1, w2, w3 = dataset.shape
#         bern = torch.bernoulli((1-p) * torch.ones((w1, w2, 1))).to("cuda:0")
#         return dataset * bern

In [18]:
lstms = 4
hiddensize = 40

In [19]:
chlstm = CharacterLSTM(vocabsize, lstms, hiddensize).to(device)

In [20]:
criterion = nn.BCELoss()
criterionPretraining = nn.CrossEntropyLoss()

In [21]:
epochs = 700

In [22]:
lrmin = 1e-8
lrmax = 1e-2

def getlr(lrmin, lrmax, epoch, epochs):
    b1 = lrmin
    a1 = (lrmax - b1) / (epochs/2)
    b2 = lrmax - ((-a1)*epochs/2)
    
    lr = 0
    
    if epoch <= epochs/2:
        return  a1*epoch+b1
    else:
        return -a1*epoch+b2

In [23]:
onecyclelambda = lambda epoch: getlr(lrmin, lrmax, epoch, epochs)

optimizerLSTM = optim.RMSprop(chlstm.parameters(), lr=1e-3)
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizerLSTM, epochs, eta_min = 1e-8)
scheduler = optim.lr_scheduler.LambdaLR(optimizerLSTM, lr_lambda=[onecyclelambda])

In [24]:
def pokrycie(zwrot, testset, device):
    d1, d2 = zwrot.shape
    
    cos = nn.CosineSimilarity(dim=1)
    
    wynik = torch.Tensor([]).to(device)
    
    indxs = [cnt for cnt in range(d1)]
    indx = [indxs[-1]]+indxs[:-1]
    
    for ii in range(d1):
        pod = cos(zwrot, testset)
        zwrot = torch.index_select(zwrot, 0, torch.Tensor(indx).long().to(device))
        wynik = torch.cat((wynik, pod), 0)
    return torch.sum(wynik.eq(1),0)

In [26]:
def accuracy(model, testsetloader, device, lstms, vocabsize):
    t_batch = tqdm_notebook(testsetloader, leave=False)
    hiddens1 = torch.randn(2*lstms, 1, vocabsize).to(device)
    
    suma = 0
    
    for batch in t_batch:
        d1, d2 = batch[0].shape

        xreals = batch[0].long().to(device)
        xfakes = torch.Tensor([]).long().to(device)
        #y = batch[1].long().cuda()
        
        ## GENEROWANIE FAKE PROBEK
        with torch.no_grad():
            chlstm.eval()

            xfake = torch.Tensor([[chartoidx["<START>"]]]).long().to(device).repeat(d1, 1)
            hidd = hiddens1.repeat(1, d1, 1)

            for cnt in range(d2-1):
                y, hidd = chlstm(xfake, hidd, drop=0)

                pred = torch.argmax(F.softmax(y.transpose(1,2), 1), 1)
                proponowanyznak = torch.index_select(pred, 1, torch.Tensor([cnt]).to(device).long())

                xfake = torch.cat((xfake, proponowanyznak.reshape(d1, 1)), 1)
            xfakes = xfake
            
            suma = suma + pokrycie(xfakes.float(), xreals.float(), device).item()
            t_batch.set_description("Acc: {:.8f}".format(suma/d1))
    t_batch.close()
    return suma/len(testsetloader)

In [24]:
# torch.save(losssess, "../wykresy/2lstm_60hidden_500epochs_onecycle1e-6-1e-2_optim1e-3.pt")

In [25]:
t_epochs = tnrange(epochs)

losssess = []

for epoch in t_epochs:
    t_batch = tqdm_notebook(DL_train, leave=False)
    
    for batch in t_batch:
        d1, d2 = batch[0].shape
        xreals = batch[0].long().to(device)
        y = batch[1].long().to(device)
        
        hiddens1 = torch.zeros(2*lstms, d1, hiddensize).to(device)
        
        chlstm.train()
        optimizerLSTM.zero_grad()
        
        y_, _ = chlstm(xreals, hiddens1, NLL=False)
#         print(y_[:,-1], torch.softmax(y_[:,-1], 1))
        loss = criterionPretraining(y_[:,-1], y.view(-1))
        
        loss.backward()
        optimizerLSTM.step()
        
        losss = loss.item()
        losssess.append(losss)
        
        t_batch.set_description("Loss: {:.8f}".format(losss))
        

    t_batch.close()
    t_epochs.set_description("Epoch {}/{}".format(epoch+1, epochs))
    scheduler.step()
    
    if epoch % 100 == 0:
        torch.save(chlstm, "../models/bezrelu_lstm_4_hidden_40_onecycle_1e-2-1e-8_rmsprop1e-3_700epoch_loss_"+str(losss)+".pt")
    
#     acc = accuracy(chlstm, DL_test, device, lstms, vocabsize)
    print("Epoch {}/{}, Loss {:.8f}, LR {:.8f}".format(epoch+1, epochs, losss, scheduler.get_lr()[0]))

Epoch 1/700, Loss 4.49691772, LR 0.00000003


/home/mborzymowski/.conda/envs/mborzymowski/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type CharacterLSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 2/700, Loss 4.49710178, LR 0.00000006


Epoch 3/700, Loss 4.49693775, LR 0.00000009


Epoch 4/700, Loss 4.49684048, LR 0.00000011


...

Epoch 696/700, Loss 2.68315196, LR 0.00000011


Epoch 697/700, Loss 2.68396497, LR 0.00000009


Epoch 698/700, Loss 2.68355894, LR 0.00000006


Epoch 699/700, Loss 2.68304706, LR 0.00000003


Epoch 700/700, Loss 2.68378854, LR 0.00000000



In [26]:
torch.save(losssess, "../wykresy/bezrelu_lstm_4_hidden_40_onecycle_1e-2-1e-8_rmsprop1e-3_700epoch_loss_"+str(losss)+".pt")

In [27]:
torch.save(chlstm, "../models/bezrelu_lstm_4_hidden_40_onecycle_1e-2-1e-8_rmsprop1e-3_700epoch_loss_"+str(losss)+".pt")

In [41]:
chardict = list(chartoidx.keys())
temperature = 0.35

# plikhasel = open("../haslaAI_bezrelu.txt", "a", encoding="utf8")

for _ in tqdm_notebook(range(20)):
    slowostart = "h"
    hiddens = torch.zeros(2*lstms, 1, hiddensize).to(device)

    with torch.no_grad():
        lastchar = 0
        cnt = 0

        chlstm.eval()
        znaki = [chartoidx[item] for item in slowostart]
        bazastart = len(znaki)

        for __ in range(longestword - bazastart+1):
            znaki.append(chartoidx["<EMPTY>"])

        for item in range(longestword - bazastart+1):
            x = torch.Tensor(znaki).long().to(device).view(1, -1)

            out, _ = chlstm(x, hiddens, NLL=False)

            zwrot = out[:,-1].view(-1)
            exped = zwrot.data.div(temperature).exp()
            top_i = torch.multinomial(exped, 1)
    #         softmaxed = torch.softmax(top_i, 0)
    #         charid = torch.argmax(softmaxed, 0)
            charid = top_i[0]

            znaki[item+bazastart] = charid


        slowo = ""
        for item in znaki:
            if item != 0:
                slowo+=chardict[item]

        print(slowo)
#         plikhasel.write(slowo+"\n")
# plikhasel.close()

horik102
harienar
hadana1909
hemisma1
hastata1
halaska
huromask
halon1924
hasiia19
horka192
henanera
hemaro19
honanka
hanero10
horzar19
haralera
hakateta
hola11992
hamirek
hanota19

